In [7]:
import os

# Set the path to the folder containing your data
folder_path = r'D:\Documentos\Semestre8\Aplicaciones avanzadas\AI\Proyecto\AImodel\images3\train'

# Iterate over each subfolder
for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        # Count the number of files in the subfolder
        file_count = len(os.listdir(subfolder_path))

        print(f"Folder: {subfolder} - Number of Files: {file_count}")


Folder: ECONOMIC_VEHICLE - Number of Files: 396
Folder: FAMILY_CAR - Number of Files: 333
Folder: LUXURY_CAR - Number of Files: 316
Folder: SPORTS_CAR - Number of Files: 420
Folder: SUPER_CAR - Number of Files: 404


In [40]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = r'D:\Documentos\Semestre8\Aplicaciones avanzadas\AI\Proyecto\AImodel\images\train'
test_data_dir = r'D:\Documentos\Semestre8\Aplicaciones avanzadas\AI\Proyecto\AImodel\images\test'
validation_data_dir = r'D:\Documentos\Semestre8\Aplicaciones avanzadas\AI\Proyecto\AImodel\images\validation'

total_train_samples = sum([len(files) for _, _, files in os.walk(train_data_dir)])

train_datagen = ImageDataGenerator(1./255)  

test_datagen = ImageDataGenerator(1./255)   

validation_datagen = ImageDataGenerator(1./255)  

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(75, 75),
    batch_size=100,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(75, 75),
    batch_size=400,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(75, 75),
    batch_size=20,
    class_mode='categorical')



Found 8548 images belonging to 16 classes.
Found 483 images belonging to 16 classes.
Found 475 images belonging to 16 classes.


In [47]:
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Conv2D(10, (3, 3), activation="relu", input_shape = (75,75,3)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(16,activation='sigmoid'))

model.summary()

model.compile(loss='categorical_crossentropy',
						optimizer='Adam',
						metrics=['acc'])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_942 (Conv2D)         (None, 73, 73, 10)        280       
                                                                 
 flatten_11 (Flatten)        (None, 53290)             0         
                                                                 
 dense_22 (Dense)            (None, 512)               27284992  
                                                                 
 dense_23 (Dense)            (None, 16)                8208      
                                                                 
Total params: 27,293,480
Trainable params: 27,293,480
Non-trainable params: 0
_________________________________________________________________


In [48]:
history = model.fit_generator(train_generator, 
				steps_per_epoch =total_train_samples/100,  
				epochs =10,
                validation_data=validation_generator,
            	validation_steps=8 
				)

C:\Users\joako\AppData\Local\Temp\ipykernel_26980\936354016.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/10
85/85 [==============================] - 47s 535ms/step - loss: 2473.4917 - acc: 0.0729 - val_loss: 15.0837 - val_acc: 0.1063
Epoch 2/10
85/85 [==============================] - 46s 542ms/step - loss: 10.2368 - acc: 0.1554 - val_loss: 10.7250 - val_acc: 0.1125
Epoch 3/10
85/85 [==============================] - 46s 542ms/step - loss: 5.7117 - acc: 0.3066 - val_loss: 10.9260 - val_acc: 0.1437
Epoch 4/10
85/85 [==============================] - 46s 539ms/step - loss: 3.0866 - acc: 0.5185 - val_loss: 6.8109 - val_acc: 0.1813
Epoch 5/10
85/85 [==============================] - 46s 544ms/step - loss: 1.3192 - acc: 0.6954 - val_loss: 6.5049 - val_acc: 0.2000
Epoch 6/10
85/85 [==============================] - 46s 540ms/step - loss: 0.5875 - acc: 0.8441 - val_loss: 5.8863 - val_acc: 0.2500
Epoch 7/10
85/85 [==============================] - 47s 551ms/step - loss: 0.3674 - acc: 0.9112 - val_loss: 6.4139 - val_acc: 0.2250
Epoch 8/10
85/85 [==============================] - 47s 554ms/

In [49]:
test_loss, test_acc = model.evaluate(test_generator, steps = 1)
print('\ntest acc :\n', test_acc)

1/1 [==============================] - 1s 1s/step - loss: 7.5171 - acc: 0.2629

test acc :
 0.2629399597644806


In [50]:
import numpy as np
test_imgs = test_generator[0][0]
test_labels = test_generator[0][1]


predictions = model.predict(test_imgs)
classes_x = np.argmax(predictions,axis=1)
classex_y = np.argmax(test_labels,axis=1)

from tensorflow.math import confusion_matrix
mat = confusion_matrix(classes_x, classex_y)
print(mat) 

16/16 [==============================] - 0s 16ms/step
tf.Tensor(
[[ 7  4  4  1  2  3  3  2  3  2  1  4  1  1  3  1]
 [11 23 10 13 18  9  7 15 14 15 19 10 11 11 17 16]
 [ 1  0  3  0  1  1  0  0  0  3  2  0  1  0  0  3]
 [ 1  1  0  4  1  1  2  0  0  1  1  1  2  1  5  0]
 [ 0  1  1  1  4  1  0  1  1  3  2  0  1  1  1  2]
 [ 0  1  1  1  2  8  2  1  3  1  0  0  1  1  0  0]
 [ 0  2  0  0  1  2  8  3  4  1  2  1  1  0  2  0]
 [ 1  0  0  0  1  0  2  5  0  1  2  0  0  1  1  0]
 [ 1  0  0  0  1  0  0  0  2  0  1  1  0  0  1  0]
 [ 0  0  0  0  0  0  1  0  2  2  1  0  2  0  0  1]
 [ 0  1  0  0  0  1  0  0  0  0  3  2  0  0  1  2]
 [ 1  0  0  1  1  4  3  0  0  2  2  3  2  0  1  0]
 [ 1  1  1  1  0  0  0  2  0  1  0  3  5  2  0  0]
 [ 1  0  1  1  0  1  0  0  3  1  0  0  1  2  3  0]
 [ 0  0  1  0  1  0  1  2  1  1  0  1  3  0  2  0]
 [ 2  1  0  1  0  1  1  0  2  0  0  0  1  2  0  2]], shape=(16, 16), dtype=int32)


In [61]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import Model
pretrained_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(75,75,3))
pretrained_model.trainable = False 

last_layer = pretrained_model.get_layer('mixed9')
last_output = last_layer.output

model2 = layers.Flatten()(last_output)
model2 = layers.Dense(1024,activation='relu')(model2)
model2 = layers.Dropout(0.2)(model2)
model2 = layers.Dense(16,activation='softmax')(model2)

model_combined = Model(pretrained_model.input, model2)

model_combined.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 75, 75, 3)]  0           []                               
                                                                                                  
 conv2d_1225 (Conv2D)           (None, 37, 37, 32)   864         ['input_14[0][0]']               
                                                                                                  
 batch_normalization_1222 (Batc  (None, 37, 37, 32)  96          ['conv2d_1225[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_1222 (Activation)   (None, 37, 37, 32)   0           ['batch_normalization_1222[

In [62]:
model_combined.compile(loss='categorical_crossentropy',
						optimizer='Adam',
						metrics=['acc'])

In [65]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True)
history2 = model_combined.fit(train_generator, 
				    steps_per_epoch =total_train_samples/100,  
				    epochs =50,
                    validation_data=validation_generator,
            	    validation_steps=8,
                    callbacks=[early_stopping]
                )

Epoch 1/50
85/85 [==============================] - 45s 530ms/step - loss: 2.4284 - acc: 0.2092 - val_loss: 2.8681 - val_acc: 0.0750
Epoch 2/50
85/85 [==============================] - 44s 516ms/step - loss: 2.3925 - acc: 0.2197 - val_loss: 2.8491 - val_acc: 0.1437
Epoch 3/50
85/85 [==============================] - 43s 506ms/step - loss: 2.3466 - acc: 0.2375 - val_loss: 2.8365 - val_acc: 0.1063
Epoch 4/50
85/85 [==============================] - 43s 505ms/step - loss: 2.2959 - acc: 0.2492 - val_loss: 3.1231 - val_acc: 0.1250
Epoch 5/50
85/85 [==============================] - 43s 503ms/step - loss: 2.2560 - acc: 0.2615 - val_loss: 2.9041 - val_acc: 0.1437
Epoch 6/50
85/85 [==============================] - 43s 509ms/step - loss: 2.2199 - acc: 0.2771 - val_loss: 2.8787 - val_acc: 0.1187
Epoch 7/50
85/85 [==============================] - 44s 520ms/step - loss: 2.2006 - acc: 0.2802 - val_loss: 3.2370 - val_acc: 0.0938
Epoch 8/50
85/85 [==============================] - 45s 525ms/step - 

In [66]:
test_loss, test_acc = model_combined.evaluate(test_generator, steps = 1)
print('\ntest acc :\n', test_acc)

1/1 [==============================] - 2s 2s/step - loss: 2.8221 - acc: 0.1242

test acc :
 0.12422360479831696


In [67]:
from sklearn.metrics import classification_report
import numpy as np
test_imgs = test_generator[0][0]
test_labels = test_generator[0][1]

class_names = list(train_generator.class_indices.keys())

predictions = model_combined.predict(test_imgs)
classes_x = np.argmax(predictions,axis=1)
classex_y = np.argmax(test_labels,axis=1)

from tensorflow.math import confusion_matrix
mat = confusion_matrix(classes_x, classex_y)
print(mat) 
print(classification_report(classes_x, classex_y, target_names=class_names))

16/16 [==============================] - 2s 68ms/step
tf.Tensor(
[[ 2  1  0  1  1  1  0  0  0  0  1  0  0  1  0  0]
 [ 2  4  0  0  4  0  1  2  1  2  0  1  1  1  0  0]
 [ 1  2  3  1  0  1  1  2  1  0  2  0  2  2  1  2]
 [ 0  0  0  0  0  0  0  1  0  0  1  0  1  1  2  0]
 [ 0  2  1  1  3  1  0  2  0  0  2  1  1  1  2  1]
 [ 1  1  1  1  3  6  1  1  2  2  4  5  2  2  2  1]
 [ 3  1  2  0  0  3  9  3  5  2  1  2  3  3  5  0]
 [ 1  1  0  2  1  1  1  3  0  0  1  0  3  0  2  0]
 [ 3  3  1  3  3  1  4  0  6  8  5  1  2  1  6  1]
 [ 9  6  7  7 11  8  5  5 11  9  7  3 11  4  5  9]
 [ 3  6  4  3  2  4  5  5  6  5  4  6  1  5  3  5]
 [ 1  1  1  0  1  0  0  0  0  0  1  3  0  1  2  1]
 [ 0  4  0  1  1  1  1  0  0  1  2  0  2  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  3]
 [ 1  3  1  4  3  4  1  4  3  4  3  4  2  0  5  3]
 [ 0  0  1  0  0  1  1  3  0  1  2  0  1  0  0  1]], shape=(16, 16), dtype=int32)
              precision    recall  f1-score   support

        Audi       0.07      0.25